In [26]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.base import TransformerMixin
import datetime

pd.set_option('display.max_columns', None)


In [27]:
raw_accepted_df = pd.read_json('accepted.json')
raw_accepted_df['accepted'] = 1
raw_accepted_df.head()

,id,name,discussions_count,comments_count,submitter,movie_count,screenshot_count,awards_count,link_count,file_size_mb,posted_date,updated_date,change_notes,certified_compatible,creator_count,featured_collection_count,visitors_count,favorites_count,class,item_slot,other,description,accepted
0,100054607,Spooky Sleeves,0,154,GT500&gt;JZeeba ジェイシーバ,0,6,0,0,4.971,"30 Sep, 2012 @ 9:39am","7 Mar, 2016 @ 10:06am",7,1,2,115,5492,428,"[demoman, engineer, heavy, scout, sniper]",[misc],[certified compatible],<type good description here>Halloween 2012 Sub...,1
1,100055144,Spooky Shoes,0,40,GT500&gt;JZeeba ジェイシーバ,0,4,0,0,2.958,"30 Sep, 2012 @ 9:43am","22 Oct, 2015 @ 8:08am",4,1,2,46,2218,173,[scout],[misc],[certified compatible],<type good description here>Halloween 2012 Sub...,1
2,100098706,The Crossed Stitch,0,238,Sparkwire,0,2,0,0,4.277,"30 Sep, 2012 @ 2:18pm",NaN,1,0,4,100,11905,859,[spy],[misc],NaN,I have no mouth and I must scream.Thanks to Tr...,1
3,100148654,Head-Takers Hallmark,0,237,&gt;&gt;-The Heartsman--&gt;,0,8,0,0,4.567,"30 Sep, 2012 @ 6:44pm","10 Oct, 2012 @ 10:52am",3,0,2,91,10085,650,"[demoman, engineer, heavy, medic, pyro, scout,...",[misc],NaN,A skull collector. A Noggin connoisseur. Put t...,1
4,10043,Engagement Ring,3,773,Ko,0,0,0,0,0.572,"11 Feb, 2012 @ 11:18am",NaN,1,0,1,17,33200,102,NaN,[misc],NaN,An engagement ring you can use to propose to t...,1


In [28]:
raw_not_accepted_df= pd.read_json('not_accepted.json')
raw_not_accepted_df['accepted'] = 0
raw_not_accepted_df.head()

,id,name,discussions_count,comments_count,submitter,movie_count,screenshot_count,awards_count,link_count,file_size_mb,posted_date,updated_date,change_notes,certified_compatible,creator_count,featured_collection_count,visitors_count,favorites_count,class,item_slot,description,other,tags,game_mode,accepted
0,100,Singed Scythe,1,143,Segab,0,2,0,0,1.885,"13 Oct, 2011 @ 4:40pm","30 Oct, 2011 @ 5:04pm",2,0,1,19,8932,221,[pyro],[weapon],This is an Halloween themed scythe for the Pyr...,NaN,NaN,NaN,0
1,100001162,The Bride,0,23,Brainsample,0,3,0,0,3.942,"30 Sep, 2012 @ 3:46am","17 Oct, 2013 @ 1:22am",2,1,1,4,1593,89,[pyro],[headgear],"The original Bride, not that one in a yellow j...","[certified compatible, halloween]",NaN,NaN,0
2,100026174,Phantasmal Perception,0,92,Mr. Sol,1,3,0,0,2.850,"30 Sep, 2012 @ 6:13am","19 Sep, 2013 @ 10:57am",4,1,1,36,5624,614,"[engineer, pyro]",[misc],Watch the embedded Youtube video for a visual ...,"[certified compatible, halloween]",NaN,NaN,0
3,100042018,The Wardens Gaze,0,54,CoreVixen,0,2,0,0,4.136,"30 Sep, 2012 @ 8:02am",NaN,1,0,2,21,4465,385,[pyro],[misc],Every murderous miner needs a modified mining ...,NaN,NaN,NaN,0
4,100043151,The Searchlight,2,30,CoreVixen,0,2,0,0,4.269,"30 Sep, 2012 @ 8:11am",NaN,1,0,2,29,3467,309,[pyro],[headgear],Mining lights are great for both tracking down...,NaN,NaN,NaN,0


In [37]:
raw_combined_df  = pd.concat([raw_accepted_df, raw_not_accepted_df], ignore_index=True, sort=False)

raw_combined_df.head()

0      13671
1       3649
2       1743
3        901
4        516
       ...  
113        1
61         1
68         1
92         1
141        1
Name: discussions_count, Length: 76, dtype: int64

In [30]:
def normalize_column(df:pd.DataFrame,column:str,scaler:TransformerMixin):
    df[column] = scaler.fit_transform(np.array(df[column]).reshape(-1, 1))

In [47]:
def prep_df(df: pd.DataFrame):
    copy = df.copy()

    copy['is_medal'] = (copy['name'].str.contains('medal',case=False) | copy['name'].str.contains('description',case=False)).astype('int')

    copy = copy.drop(columns=['name','description','id','tags', 'game_mode','other'], errors='ignore')

    # copy['description'] = copy['description'].str.split().map(lambda words: len(words))

    copy['updated_date'].fillna(copy['posted_date'],inplace=True)

    copy['posted_time'] = copy['posted_date'].map(lambda ds: ds.replace(' @',', 2022 @') if ',' not in ds else ds).map(lambda ds:datetime.datetime.strptime(ds,'%d %b, %Y @ %I:%M%p').timestamp())
    copy['updated_time'] = copy['updated_date'].map(lambda ds: ds.replace(' @',', 2022 @') if ',' not in ds else ds).map(lambda ds:datetime.datetime.strptime(ds,'%d %b, %Y @ %I:%M%p').timestamp())

    copy['passed_time'] = pd.cut(copy['updated_time'] - copy['posted_time'],bins=10,labels=range(0,10))

    copy['posted_time'] = copy['posted_time'].map(lambda ms:datetime.datetime.fromtimestamp(ms).year -2012)
    copy['updated_time'] = copy['updated_time'].map(lambda ms:datetime.datetime.fromtimestamp(ms).year -2012)
    
    copy['media_count'] = copy['movie_count'] + copy['screenshot_count'] 

    copy['class'] = copy['class'].fillna("").apply(list)
    copy['item_slot'] = copy['item_slot'].fillna("").apply(list)


    copy['class_count'] = copy['class'].map(lambda arr:len(arr))
    copy['item_slot_count'] = copy['item_slot'].map(lambda arr:len(arr))
    #copy['other'] = copy['other'].fillna("").apply(list)

    classes=['scout', 'soldier', 'pyro', 'demoman', 'heavy', 'engineer', 'medic', 'sniper', 'spy']
    item_slots = list(set(copy['item_slot'].sum()))
    item_slots.sort()
    #others = list(set(copy['other'].sum()))

    submitter_value_counts = copy['submitter'].value_counts()

    copy['submitted_by_submitter'] =  copy['submitter'].map(lambda s:submitter_value_counts[s])

    for c in classes:
        copy[f'class_{c}']= copy['class'].map(lambda arr: c in arr).astype('int')

    for s in item_slots:
        copy[f'item_slot_{s}']= copy['item_slot'].map(lambda arr: s in arr).astype('int')

    copy['v1'] = (copy['featured_collection_count'] / copy['favorites_count']).fillna(0)
    copy['v2'] = (copy['comments_count'] / (copy['discussions_count']+1)).fillna(0)
    copy['v3'] = (copy['visitors_count'] / (copy['discussions_count']+1)).fillna(0)
    copy['v4'] = (copy['visitors_count'] / (copy['comments_count']+1)).fillna(0)
    copy['v5'] = (copy['screenshot_count'] / (copy['media_count']+1)).fillna(0)

    copy['v6'] = (copy['class_spy'] == copy['class_pyro']).astype('int')
    copy['v7'] = (copy['class_engineer'] == copy['class_spy']).astype('int')
    copy['v8'] = (copy['item_slot_weapon'] == copy['item_slot_headgear']).astype('int')
      
    normalize_column(copy,'comments_count',StandardScaler())
    normalize_column(copy,'visitors_count',StandardScaler())
    normalize_column(copy,'discussions_count',StandardScaler())

    copy = copy.drop(columns=['class', 'item_slot', 'other','submitter','posted_date','updated_date'],errors='ignore')

    print(f"shape {copy.shape}")
    print(f"non number {len(copy.select_dtypes(exclude=['number','category']).columns)}")
    print(f"na cells {copy.isna().sum().sum()}")

    return copy

In [48]:
prepped_df = prep_df(raw_combined_df)

prepped_df.head()

shape (22100, 43)
non number 0
na cells 0


,discussions_count,comments_count,movie_count,screenshot_count,awards_count,link_count,file_size_mb,change_notes,certified_compatible,creator_count,featured_collection_count,visitors_count,favorites_count,accepted,is_medal,posted_time,updated_time,passed_time,media_count,class_count,item_slot_count,submitted_by_submitter,class_scout,class_soldier,class_pyro,class_demoman,class_heavy,class_engineer,class_medic,class_sniper,class_spy,item_slot_headgear,item_slot_misc,item_slot_taunt,item_slot_weapon,v1,v2,v3,v4,v5,v6,v7,v8
0,-0.299559,0.387851,0,6,0,0,4.971,7,1,2,115,-0.003128,428,1,0,0,4,3,6,5,1,380,1,0,0,1,1,1,0,1,0,0,1,0,0,0.268692,154.00,5492.0,35.432258,0.857143,1,0,1
1,-0.299559,-0.119405,0,4,0,0,2.958,4,1,2,46,-0.261479,173,1,0,0,3,2,4,1,1,380,1,0,0,0,0,0,0,0,0,0,1,0,0,0.265896,40.00,2218.0,54.097561,0.800000,1,1,1
2,-0.299559,0.761618,0,2,0,0,4.277,1,0,4,100,0.502919,859,1,0,0,0,0,2,1,1,153,0,0,0,0,0,0,0,0,1,0,1,0,0,0.116414,238.00,11905.0,49.811715,0.666667,0,0,1
3,-0.299559,0.757168,0,8,0,0,4.567,3,0,2,91,0.359304,650,1,0,0,0,0,8,9,1,114,1,1,1,1,1,1,1,1,1,0,1,0,0,0.140000,237.00,10085.0,42.373950,0.888889,1,1,1
4,0.320877,3.142160,0,0,0,0,0.572,1,0,1,17,2.183300,102,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0.166667,193.25,8300.0,42.894057,0.000000,1,1,1


In [49]:
prepped_df.to_csv('final_data.csv',index=False)